In [371]:
import pandas as pd

In [398]:
words = open('indian_names.txt','r').read().splitlines()
words = [word.lower() for word in words]

In [399]:
vocab = ['.'] + sorted(list(set(''.join(words))))
stoi = {v:i for i,v in enumerate(vocab)}
itos = {v:k for k,v in stoi.items()}

In [435]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback
import pandas as pd

In [401]:
# Tokenize the names at character level
# words = [word + '.' for word in words]
sequences = []
for word in words:
    seq = [stoi[c] for c in word]
    sequences.append(seq)
# Create input sequences and labels
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])
    X.append(seq)
    y.append(0)
    
# Pad sequences for consistent input shape
max_seq_length = max([len(seq) for seq in X])
X = pad_sequences(X, maxlen=max_seq_length, padding='pre')

# One-hot encode labels
y = tf.keras.utils.to_categorical(y, num_classes=len(stoi) + 1)

In [402]:
from sklearn.model_selection import train_test_split

In [403]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [436]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(len(stoi) + 1, 32, input_length=max_seq_length))
model.add(LSTM(64))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(stoi) + 1, activation='softmax'))

In [438]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=20)

Epoch 1/10
298/298 [==============================] - 31s 27ms/step - loss: 2.0583 - val_loss: 2.0407
Epoch 2/10
298/298 [==============================] - 7s 25ms/step - loss: 2.0444 - val_loss: 2.0188
Epoch 3/10
298/298 [==============================] - 8s 25ms/step - loss: 2.0247 - val_loss: 2.0140
Epoch 4/10
298/298 [==============================] - 7s 24ms/step - loss: 2.0189 - val_loss: 2.0060
Epoch 5/10
298/298 [==============================] - 7s 24ms/step - loss: 2.0108 - val_loss: 2.0028
Epoch 6/10
298/298 [==============================] - 7s 24ms/step - loss: 1.9887 - val_loss: 1.9975
Epoch 7/10
298/298 [==============================] - 7s 24ms/step - loss: 1.9824 - val_loss: 1.9910
Epoch 8/10
298/298 [==============================] - 7s 24ms/step - loss: 1.9694 - val_loss: 1.9844
Epoch 9/10
298/298 [==============================] - 7s 23ms/step - loss: 1.9515 - val_loss: 1.9793
Epoch 10/10
298/298 [==============================] - 7s 24ms/step - loss: 1.9393 - val_l

In [421]:
def generate_name(model,temperature):
    seed_text = itos[np.random.randint(0,27)]
    generated_text = ""
    
    while True:
        sequence = [stoi[c] for c in seed_text]
        sequence = pad_sequences([sequence], maxlen=max_seq_length, padding='pre')

        predicted_probabilities = model.predict(sequence)
        predicted_probabilities/=temperature
        predicted_id = tf.random.categorical(
        predicted_probabilities,
        num_samples=1
        )[-1,0].numpy()
        if predicted_id == len(stoi):
            continue
        if predicted_id == 0:
            break
        generated_text += itos[predicted_id]
        seed_text += itos[predicted_id]
    return generated_text

In [443]:
names = []
for i in range(10):
    names.append(generate_name(model,0.06))

1/1 [==============================] - 0s 31ms/step


In [444]:
names

['ranal',
 'arari',
 'haaya',
 'apan',
 'aman',
 'anish',
 'anhan',
 'ashi',
 'aixa',
 'ika']